In [1]:
import json
import boto3

In [ ]:
def send_survey(survey_path, sqs_url):
    '''
        Input:
            survey_path (str): path to JSON survey data (e.g. ‘./survey.json’)
            sqs_url (str): URL for SQS queue
        Output:
            StatusCode (int): indicating whether the survey was successfully 
            sent into the SQS queue (200) or not (0)
    '''
    # Load survey data from the file
    with open(survey_path, 'r') as file:
        survey_data = json.load(file)

    # Create a boto3 client for SQS
    sqs_client = boto3.client('sqs')

    # Send the message to the SQS queue
    try:
        response = sqs_client.send_message(
            QueueUrl=sqs_url,
            MessageBody=json.dumps(survey_data)
        )
        return 200  # Successful status code
    except Exception as e:
        print(f"Failed to send message: {e}")
        return 0  # Failure status code

In [ ]:
import boto3

aws_lambda = boto3.client('lambda')
iam_client = boto3.client('iam')

role = iam_client.get_role(RoleName='LabRole')

# Initialize S3 client
s3_client = boto3.client('s3')

# Define the bucket name (must be globally unique)
bucket_name = 'a2q1-bucket'

# Create the S3 bucket
s3_client.create_bucket(Bucket=bucket_name)
print(f"S3 bucket {bucket_name} created successfully.")

# Initialize a DynamoDB client
dynamodb = boto3.resource('dynamodb')

# Define the table name
table_name = 'a2q1-table'

try: 
    # Create the DynamoDB table
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[
            {'AttributeName': 'user_id', 'KeyType': 'HASH'},  # Partition key
            {'AttributeName': 'timestamp', 'KeyType': 'RANGE'}  # Sort key
        ],
        AttributeDefinitions=[
            {'AttributeName': 'user_id', 'AttributeType': 'S'},
            {'AttributeName': 'timestamp', 'AttributeType': 'S'}
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 1,
            'WriteCapacityUnits': 1
        }
    )
    table.meta.client.get_waiter('table_exists').wait(TableName=table_name)
    print(f"DynamoDB table {table_name} created successfully.")
except Exception as e:
    print(f"Error creating DynamoDB table: {str(e)}")


with open('q1c_lambda.zip', 'rb') as f:
    lambda_zip = f.read()

try:
    response = aws_lambda.create_function(
        FunctionName='a2q1_lambda',
        Runtime='python3.11',
        Role=role['Role']['Arn'],
        Handler='q1c_lambda.lambda_handler',
        Code=dict(ZipFile=lambda_zip),
        Timeout=300
)
except aws_lambda.exceptions.ResourceConflictException:
    response = aws_lambda.update_function_code(
        FunctionName='a2q1_lambda',
        ZipFile=lambda_zip
        )
print(response)

response = aws_lambda.put_function_concurrency(
        FunctionName='a2q1_lambda',
        ReservedConcurrentExecutions=10
    )
print(response)

In [ ]:
import boto3

# Create an SQS client
sqs_client = boto3.client('sqs')

# Specify the name of the queue
queue_name = 'a2q1-queue'

# Get the URL of the queue
queue_url_response = sqs_client.get_queue_url(QueueName=queue_name)
queue_url = queue_url_response['QueueUrl']

# Get the queue ARN using the queue URL
queue_attributes_response = sqs_client.get_queue_attributes(
    QueueUrl=queue_url,
    AttributeNames=['QueueArn']
)
queue_arn = queue_attributes_response['Attributes']['QueueArn']

sqs_client.set_queue_attributes(
    QueueUrl=queue_url,
    Attributes={'VisibilityTimeout': '1800'}
)

# Create a Lambda client
lambda_client = boto3.client('lambda')

response = lambda_client.create_event_source_mapping(
    FunctionName='a2q1_lambda',
    EventSourceArn=queue_arn,
    Enabled=True,
    BatchSize=10
)

In [ ]:
import boto3
from q1a_func import send_survey
import os


# Create an SQS client
sqs_client = boto3.client('sqs')

# Specify the name of your queue
queue_name = 'a2q1-queue'

# Get the URL of the queue
queue_url_response = sqs_client.get_queue_url(QueueName=queue_name)
sqs_url = queue_url_response['QueueUrl']

json_directory = './test_json'


# Iterate over each file in the directory and send it to the SQS queue
for filename in os.listdir(json_directory):
    if filename.endswith('.json'):
        file_path = os.path.join(json_directory, filename)
        status_code = send_survey(file_path, sqs_url)
        print(f"File: {filename}, Status Code: {status_code}")

# This script will need to be run in an environment with network access to AWS

In [ ]:
import boto3

# Create an SQS client
sqs_client = boto3.client('sqs')

# Specify the name of your queue
queue_name = 'a2q1-queue'

# Get the URL of the queue
queue_url_response = sqs_client.get_queue_url(QueueName=queue_name)
sqs_url = queue_url_response['QueueUrl']


# Delete the SQS queue
response = sqs_client.delete_queue(QueueUrl=sqs_url)
print(response)

# Also delete S3 and DynamoDB manually